In [1]:
import os
import time
from pynput.keyboard import Key, Controller
import threading

keyboard = Controller()
results = None


# _write_test_program - reads into memory, the program undertests
# modifies the program by adding function and return statements, and writes
# to disk a new program (named _prg_undr_test.py) that can be imported
# and called by _prog_driver.py
# 	[code is too complex; will need to introduce regex, and reduce complexity, 
# 	and a number of unit tests]
# maintained by kalen.howell@gmail.com 

def to_funct(fpath, file_name, return_results, cr_func_w_main=False):        
    
    #fpath='/Users/kalenhowellsr/Downloads/Debugging_Exercise_2'
    #file_name = 'Debug1.py'
    file_name=os.path.join(fpath, file_name)
    
    
    if return_results != '':
        cr_return=True
    else:
        cr_return=False
    with open(file_name, "r") as in_file:
        buf = in_file.readlines()
    cnt = 0
    if cr_func_w_main == False and cr_return==True:
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                if cnt == 0:
                    line = "def do_work():\n\t" + line
                else:
                    line = "\t" + line
                cnt =+1
                out_file.write(line)
            out_file.write("\n\treturn " + return_results)
    elif cr_func_w_main == True:
        in_funct = False
        ret_writen = False
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                if cr_return ==True and "def " in line and in_funct == True:
                    line = "    return " + return_results + "\n" + line
                    in_funct = False
                    ret_writen = True
                if "def main():" in line:
                    line = "def do_work():\n"
                    in_funct = True
                if "main()" in line:
                    line = ''
                out_file.write(line)
            if ret_writen == False:
                out_file.write("\n    return " + return_results)
    else:
        with open("_prg_undr_test" + ".py", "w") as out_file:
            for line in buf:
                out_file.write(line)


In [2]:
# _prog_driver.py - is a multithreaded application used to: 
# 1. launch the program under tests
# 2. drive data entry of the program under tests
# 3. compares expted results against actual results of the program under tests
# maintained by kalen.howell@gmail.com
#import _prg_undr_test as prog

def enter(data):
    for d in data:
        keyboard.press(d)
        keyboard.release(d)
    keyboard.press(Key.enter)
    keyboard.release(Key.enter)
    time.sleep(1)

def launch_typer(input_data):
    time.sleep(1)
    for d in input_data:
        enter(d)

def launch_prg(prog, expected_results):
    results = prog.do_work()
    try:
        assert results == expected_results, "\n **ERROR** \texpecting: " + str(expected_results) + "\n\trecieved: " + str(results) +"\n"
    except Exception as e: print(e)
    
def test(prog, input_data, expected_results):
    
    prog_ut = threading.Thread(target=launch_prg, args=(prog, expected_results,),daemon=True)
    typer = threading.Thread(target=launch_typer, args=(input_data,), daemon=True)

    prog_ut.start()
    typer.start()
    prog_ut.join()

### Cookies Tests

In [27]:
# pygrader_test.py - this is where the specific tests are configured. 

# tp.to_funct() - reads the program under test, and writes a new program, adding
#		necessary functions that are used to drive the program. 
# fpath - specify the path where the program under tests resides
# file_name - the name of the program to test
# return_results - the name of the primary function; returns results of prog execution  
# cr_func_w_main - True - indicates the program has main() function
#		   False - indicates the program has no function. so create one
to_funct(fpath='/Users/kalenhowellsr/Projects/code/pygrader/test_dir/',
            file_name='Cookies.py',
            return_results='sugarNeeded, butterNeeded, flourNeeded',
            cr_func_w_main=False)

# after the new program is written, it is imported and driven by _prog_driver
import _prg_undr_test as prog

# input_data - the data to be entered into the running program under tests. 
# expected_results - the expected results to be returned by the program under tests. 
test(prog, input_data=['24'] , expected_results= (0.75, 0.5, 1.375))
test(prog, input_data=['65'] , expected_results= (2.03125, 1.3541666666666665, 3.723958333333333))

Enter the number of cookies: 24
The amount of sugar needed is: 0.750 cups
The amount of butter needed is: 0.500 cups
The amount of flour needed is: 1.375 cups
Enter the number of cookies: 65
The amount of sugar needed is: 2.031 cups
The amount of butter needed is: 1.354 cups
The amount of flour needed is: 3.724 cups


#### 